In [2]:
import os

import numpy as np
import pandas as pd
#from scripts.clickmodel_fitters.clickdefinitionreader import ClickDefinition
#from scripts.clickmodel_fitters.GCM import GCM
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from copy import deepcopy

os.chdir("/Users/cornederuijt/github/GCM/") # Adjust after construction of the package
np.random.seed(1992)

In [2]:
# Define the model

list_size = 10
no_states = 11
ubm_click_states = np.eye(list_size + 1, list_size + 1)
abs_state = [(i, i) for i in range(no_states)]
init_state = 0
batch_size = 10000
no_items = 100

var_dic = {
    'phi_A': {
        'var_type': 'item',
        'pos_mat': np.triu(np.ones((list_size + 1, list_size + 1)), k=1)
    },
    'gamma': {
        'var_type': 'state',
        'pos_mat': np.triu(np.ones((list_size + 1, list_size + 1)), k=1)
    },
    'tau': {
        'var_type': 'pos',
        'pos_mat': np.triu(np.ones((list_size + 1, list_size + 1)), k=1)
    }
}

model_def = ClickDefinition(ubm_click_states, init_state, list_size, no_states, batch_size, no_items, abs_state,
                            var_dic)

In [3]:
# Load data:
click_data = pd.read_csv("./data/small_example/simulation_res_train.csv", index_col=False)
prod_position = pd.read_csv("./data/small_example/simulation_item_props.csv", index_col=False)

    
# Ensure the order is correct:
click_data = click_data.sort_values(['user_id', 'session_count', 'item_order'])
    
# Add session index:
session_index = (click_data
                 .loc[:, ['user_id', 'session_count']] 
                 .drop_duplicates() 
                 .reset_index()
                )

session_index['session'] = session_index.index.to_numpy()

click_data = (click_data 
    .set_index(['user_id', 'session_count']) 
    .join(session_index
          .set_index(['user_id', 'session_count']),
          on=['user_id', 'session_count']) 
    .reset_index()
    .set_index('item')
    .join(prod_position
         .set_index('item'),
         on='item')
    .reset_index()
)

click_data



,item,user_id,session_count,item_order,click,attr,satis,eval,orig_list_id,index,session,X0,X1
0,42,0,0,1,0.0,0,0,1.0,0,0,0,-0.617880,-0.424150
1,66,0,0,2,0.0,0,0,1.0,0,0,0,-0.463804,-0.667938
2,89,0,0,3,0.0,0,0,1.0,0,0,0,-0.849402,1.813430
3,64,0,0,4,0.0,0,0,1.0,0,0,0,-1.875517,-1.026840
4,25,0,0,5,0.0,0,0,1.0,0,0,0,-0.908807,0.050236
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250565,85,17999,2,6,0.0,0,0,0.0,35807,250560,25056,-0.362179,-0.737943
250566,3,17999,2,7,0.0,1,0,0.0,35807,250560,25056,0.136692,0.039374
250567,76,17999,2,8,0.0,1,0,0.0,35807,250560,25056,0.257214,-0.653226
250568,64,17999,2,9,0.0,0,0,0.0,35807,250560,25056,-1.875517,-1.026840


In [4]:
# Create the click matrix and item position matrix
click_mat = click_data.loc[:, ['session', 'item_order', 'click']] \
    .pivot(index='session', columns='item_order', values='click') \
    .to_numpy()

item_pos_mat = click_data.loc[:, ['session', 'item_order', 'item']] \
    .pivot(index='session', columns='item_order', values='item') \
    .to_numpy()

In [5]:

# Ensure the order is correct
item_feature_mat_A = (click_data.loc[:, ['item', 'X0', 'X1']] 
                                .drop_duplicates() 
                                .sort_values('item')
                                .to_numpy()
                     )

pos_feature_mat_tau_lst = []

pos_feature_mat_tau = np.vstack([np.hstack((np.zeros((model_def.list_size + 1, k1)),
            np.hstack((np.ones(k1), np.zeros(model_def.list_size - k1 + 1))).reshape(-1, 1),
            np.zeros((model_def.list_size + 1, model_def.list_size - k1)))).flatten()
 for k1 in range(1, model_def.list_size+1)])

pos_feature_gamma = np.eye(model_def.list_size+1, model_def.list_size + 1)
pos_feature_gamma[:, 0] = 0

var_dic = {'phi_A': item_feature_mat_A, 'gamma': pos_feature_gamma, 'tau': pos_feature_mat_tau}

pos_feature_mat_tau

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:

    model_phi_A = Sequential()
    model_phi_A.add(Dense(1, input_dim=var_dic['phi_A'].shape[1], activation='sigmoid', use_bias=False))
    model_phi_A.compile(loss=GCM.pos_log_loss, optimizer=RMSprop())

    # Note the large output dimension and the softmax. We want multiple transition probabilities that sum up to 1
    # Its the shape**2, as we flatten the square matrix.
    model_gamma = Sequential()
    # First compute the kernel
    model_gamma.add(Dense(var_dic['gamma'].shape[1], use_bias=False, activation=alt_softmax,
                          kernel_initializer=gamma_initializer, kernel_constraint=LowerDiagWeight()))
    model_gamma.compile(loss=GCM.pos_log_loss, optimizer=RMSprop())

    model_tau = Sequential()
    model_tau.add(Dense(var_dic['tau'].shape[1], input_dim=var_dic['tau'].shape[1], activation=None, use_bias=False,
                        kernel_initializer=Identity(), trainable=False))
    model_tau.compile('rmsprop', 'binary_crossentropy')  # No trainable weights, so doesn't really matter

    var_models = {'phi_A': model_phi_A, 'gamma': model_gamma, 'tau': model_tau}

In [ ]:
res = GCM.runEM(click_mat.to_numpy(), var_dic, var_models, item_pos_mat.to_numpy(), model_def, verbose=True)